# HCPCS-2: HCPCS is in National Institutional Outpatient file

Description: check if most of the HCPCS codes exist in the National Institutional Outpatient file.

Starting Author: Amy Jin (amy@careset.com)

Date: May 2nd, 2018

https://docs.google.com/spreadsheets/d/1IYg01IpssJaWHo6KxO4_dSDgXtYNFy41S5cIHFLvlGQ/edit#gid=604789549

## Connection to Parenthood Server

In [1]:
# Packages import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import mysql.connector
import sshtunnel
import pureyaml

# Handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml");
config = pureyaml.load(config_file.read());

# Argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# Argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

# Connect to Parenthood server
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!


## Test Function

In [2]:
# --------------------------------------- Inputs: ---------------------------------------
# 1) db_name:                database name in server
# 2）table_name:             table name
# 3) col_name:               column to test
# --------------------------------------- Outputs: --------------------------------------
# 1) The test will print out:
#       - distinct count of HCPCSs that are not in National Institutional Outpatient and
#       - total distinct number of HCPCSs.


def hcpcs_2(db_name, table_name, col_name):
    
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        print ('Test file: {}.{}'.format(db_name, table_name))
        print ('\n')
        
        # MySQL query to get distinct count of HCPCSs that are not in National Institutional Outpatient
        query = ('''
                SELECT COUNT(DISTINCT A.{col1})
                FROM {db}.{t1} AS A
                  LEFT JOIN (
                              SELECT *
                              FROM hcpcs_global_carrier.hcpcs_global_carrier_RIFQ2016
                              WHERE hcpcs_cd IS NOT NULL AND hcpcs_cd <> ''
                              UNION
                              SELECT *
                              FROM hcpcs_global_carrier.hcpcs_global_carrier_RQ17_YTD_6
                              WHERE hcpcs_cd IS NOT NULL AND hcpcs_cd <> ''
                              UNION
                              SELECT *
                              FROM hcpcs_global_carrier.hcpcs_global_carrier_RIFQ2015
                              WHERE hcpcs_cd IS NOT NULL AND hcpcs_cd <> ''
                              UNION
                              SELECT *
                              FROM hcpcs_global_carrier.hcpcs_global_carrier_16_17_6
                              WHERE hcpcs_cd IS NOT NULL AND hcpcs_cd <> ''
                              UNION
                              SELECT *
                              FROM hcpcs_global_carrier.hcpcs_global_carrier_RQ17_YTD_9
                              WHERE hcpcs_cd IS NOT NULL AND hcpcs_cd <> ''
                            ) AS B
                    ON A.{col1} = B.hcpcs_cd
                WHERE B.hcpcs_cd IS NULL;
        '''.format(db = db_name, t1 = table_name, col1 = col_name))

        cur.execute(query)
       
        print ("The distinct count of HCPCSs that are not in National Institutional Outpatient is:" + '\n')
            
        for row in cur.fetchall():
            for i in range(0,len(row)):
                print (str(row[i]))
            print ('\n')
            
        # MySQL query to get total distinct count of HCPCSs
        query = ('''
                SELECT COUNT(DISTINCT {col1})
                FROM {db}.{t1} AS A;
        '''.format(db = db_name, t1 = table_name, col1 = col_name))

        cur.execute(query)
        
        print ("The distinct count of HCPCSs in {} is:".format(table_name) )
            
        for row in cur.fetchall():
            for i in range(0,len(row)):
                print (str(row[i]))
            print ('\n')
            
        cur.close()
        connection.close()

## Test Example

In [3]:
hcpcs_2('_amy', 'test_data_good','hcpcs_cd')

Test file: _amy.test_data_good


The distinct count of HCPCSs that are not in National Institutional Outpatient is:

654


The distinct count of HCPCSs in test_data_good is:
2000




In [4]:
hcpcs_2('_amy', 'test_data_bad1','hcpcs_cd')

Test file: _amy.test_data_bad1


The distinct count of HCPCSs that are not in National Institutional Outpatient is:

199


The distinct count of HCPCSs in test_data_bad1 is:
1084


